In [1]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver


In [2]:

driver = webdriver.Chrome('E:\Programming\chromedriver') 

In [3]:
url = "https://amazon.com"
driver.get(url)

In [4]:
def get_url(search_term):
    template= 'https://www.amazon.com/s?k={}&ref=nb_sb_ss_ts-doa-p_2_6'
    search_term = search_term.replace(' ', '+')
    return template.format(search_term)


In [5]:
url = get_url('ultrawide monitor')
print(url)

https://www.amazon.com/s?k=ultrawide+monitor&ref=nb_sb_ss_ts-doa-p_2_6


In [6]:
driver.get(url)

In [7]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [8]:
results = soup.find_all('div',{'data-component-type': 's-search-result'})

In [9]:
item = results[0]
atag = item.h2.a
description = atag.text.strip()
url='https://www.amazon.com' + atag.get('href')

In [10]:
price_parent = item.find('span', 'a-price')

In [11]:
price = price_parent.find('span', 'a-offscreen').text

In [12]:
rating = item.i.text

In [13]:
review_count = item.find('span',{'class':'a-size-base'}).text

In [14]:
## Generalize the pattern##
def extract_record(item):
    ##extract and return data from a single record##
    
    ##description & url###
    atag = item.h2.a
    description = atag.text.strip()
    url='https://www.amazon.com' + atag.get('href')
    
    try:
    ##price
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    try:
    ##rank & rating
        rating = item.i.text
        review_count = item.find('span',{'class':'a-size-base'}).text
    except AttributeError:
        rating = ''
        review_count = ''
    
    result =(description, price, rating, review_count, url)
    
    return result

In [15]:
records = []
results = soup.find_all('div',{'data-component-type':'s-search-result'})
for item in results:
    record = extract_record(item)
    if record:
        records.append(record)

In [16]:
records[0]

('LG 32UP83A-W 31.5 Inch Class UHD (3840 x 2160) IPS Monitor with AMD FreeSync, DCI-P3 95% Color Gamut with HDR 10 Compatibility and USB Type-C, Tilt/Height/Pivot Stand – 2021',
 '$549.99',
 '4.6 out of 5 stars',
 '1,210',
 'https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A08728173J5KDIK28ZOV&url=%2FLG-32UP83A-W-Monitor-FreeSync-Compatibility%2Fdp%2FB09GHTPWFV%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dultrawide%2Bmonitor%26qid%3D1643009929%26sr%3D8-1-spons%26psc%3D1&qualifier=1643009929&id=3879808610983290&widgetName=sp_atf')

In [17]:
for row in records:
    print(row[1])

$549.99
$429.97
$1,199.99
$409.99
$234.97
$529.98
$1,596.99
$279.99
$266.99
$799.00
$349.99
$452.78
$299.99
$467.99
$349.99
$499.96
$179.99
$437.35


In [18]:
def get_url(search_term):
    template= 'https://www.amazon.com/s?k={}&ref=nb_sb_ss_ts-doa-p_2_6'
    search_term = search_term.replace(' ', '+')
    
    url = template.format(search_term)
    
    url += '&page{}'
    return url


In [19]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver

def get_url(search_term):
    template= 'https://www.amazon.com/s?k={}&ref=nb_sb_ss_ts-doa-p_2_6'
    search_term = search_term.replace(' ', '+')
    
    url = template.format(search_term)
    
    url += '&page{}'
    return url

def extract_record(item):
    ##extract and return data from a single record##
    
    ##description & url###
    atag = item.h2.a
    description = atag.text.strip()
    url='https://www.amazon.com' + atag.get('href')
    
    try:
    ##price
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    try:
    ##rank & rating
        rating = item.i.text
        review_count = item.find('span',{'class':'a-size-base'}).text
    except AttributeError:
        rating = ''
        review_count = ''
    
    result =(description, price, rating, review_count, url)
    
    return result

def main(search_term):
    driver = webdriver.Chrome('E:\Programming\chromedriver') 
    
    record = []
    url = get_url(search_term)
    
    for page in range(1,17):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div',{'data-component-type': 's-search-result'})
        
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
                
    driver.close()
    
    #save data to csv file
    
    with open('amazon.csv','w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['description', 'price', 'rating', 'review_count', 'url'])
        writer.writerows(records)
    

In [20]:
main('ultrawide monitor')
